In [1]:
import pandas as pd
import numpy as np

In [2]:
file_loc = '/Users/ernestvmo/Downloads/2012.csv'

In [3]:
df = pd.read_csv(file_loc)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (10,73,103,110,111) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df.head()

,year,pct,ser_num,datestop,timestop,recstat,inout,trhsloc,perobs,crimsusp,...,zip,addrpct,sector,beat,post,xcoord,ycoord,dettypcm,linecm,detailcm
0,2012,40,17,1012012,115,1,O,P,2.0,ROBBERY,...,,40,C,*,,1008031,233036,CM,1,85
1,2012,23,691,1012012,310,1,I,P,2.0,M,...,,23,D,,12,1000852,228179,CM,1,9
2,2012,81,3714,1012012,2000,1,O,P,1.0,ROBBERY,...,,81,C,3,,1001869,190702,CM,1,85
3,2012,81,633,1022012,1245,1,O,P,3.0,ROBBERY,...,,81,J,*,,1005306,186668,CM,1,85
4,2012,66,36,1042012,2220,A,O,P,2.0,FELONY,...,,66,J,,,986887,173599,CM,1,46


In [5]:
df.drop('linecm', axis=1)

,year,pct,ser_num,datestop,timestop,recstat,inout,trhsloc,perobs,crimsusp,...,state,zip,addrpct,sector,beat,post,xcoord,ycoord,dettypcm,detailcm
0,2012,40,17,1012012,115,1,O,P,2.0,ROBBERY,...,,,40,C,*,,1008031,233036,CM,85
1,2012,23,691,1012012,310,1,I,P,2.0,M,...,,,23,D,,12,1000852,228179,CM,9
2,2012,81,3714,1012012,2000,1,O,P,1.0,ROBBERY,...,,,81,C,3,,1001869,190702,CM,85
3,2012,81,633,1022012,1245,1,O,P,3.0,ROBBERY,...,,,81,J,*,,1005306,186668,CM,85
4,2012,66,36,1042012,2220,A,O,P,2.0,FELONY,...,,,66,J,,,986887,173599,CM,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532906,2012,123,1949,11042012,1930,A,O,P,1.0,FELONY,...,,,123,B,,,916798,125363,CM,14
532907,2012,123,1950,11042012,1930,A,O,P,1.0,FELONY,...,,,123,B,,,916798,125363,CM,14
532908,2012,123,1951,11042012,2305,A,O,P,2.0,FELONY,...,,,123,C,,,928562,126358,CM,14
532909,2012,123,1952,11042012,2305,A,O,P,2.0,FELONY,...,,,123,C,,,928562,126358,CM,14


In [9]:
dates = df['datestop'].values

In [10]:
dates

array([ 1012012,  1012012,  1012012, ..., 11042012, 11042012, 11082012])

In [37]:
def format_date(values):
    values = values.astype('str')
    for fulldate, _ in zip(values, range(10)):
        # print(type(date))
        year, fulldate = int(fulldate[-4:]), fulldate[:-4] 
        day, fulldate = int(fulldate[-2:]), fulldate[:-2] 
        month = int(fulldate[:])
        
        print(f'{day}, {month}, {year}')

In [38]:
format_date(dates)

1, 1, 2012
1, 1, 2012
1, 1, 2012
2, 1, 2012
4, 1, 2012
5, 1, 2012
5, 1, 2012
6, 1, 2012
6, 1, 2012
6, 1, 2012
